In [1]:
import tensorflow_datasets as tfds
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import json
from collections import defaultdict
from skimage import io
from skimage.color import rgb2gray
#from skimage.color import rgb2gray, rgb2grey
from scipy.ndimage.filters import gaussian_filter
import imageio
import math

2022-12-13 15:34:44.398301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 15:34:44.527925: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-13 15:34:45.655978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.4/lib64
2022-12-13 15:34:45.656041: W tensorflow/compil

# create imagenet 2012

In [ ]:
dataset_dir = '../../dataset_files'  # directory where you downloaded the tar files to
temp_dir = '../temp_dataset_data'

In [ ]:
download_config = tfds.download.DownloadConfig(
    extract_dir=os.path.join(temp_dir, 'extracted'),
    manual_dir=dataset_dir
)

tfds.builder("imagenet2012").download_and_prepare(download_config=download_config)

# load dataset
The loaded test dataset has all labels equal to -1. That is why the part of train dataset was taken also for test proposes

In [2]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    builder = tfds.builder("imagenet2012")
    raw_valid_ds, raw_test_ds, raw_train_ds = builder.as_dataset(['validation', 'train[0:50000]', 'train[50000:]'],as_supervised=True)

2022-12-13 15:34:52.678073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 15:34:53.815373: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30963 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2022-12-13 15:34:53.816182: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30963 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [3]:
print(len(raw_train_ds))
print(len(raw_test_ds))
print(len(raw_valid_ds))

1231167
50000
50000


In [ ]:

#builder = tfds.builder("imagenet2012")
#raw_train_ds, raw_valid_ds, raw_test_ds = builder.as_dataset(['train' , 'validation[50%:]','validation[:50%]'],as_supervised=True)

# prepare dataset

In [4]:
with open('./mapping.json') as json_file:
    subclass_to_class_int = json.load(json_file)
    
labels = builder.info.features['label'].names

tensor_keys = list(range(len(labels)))
tensor_values = list()
for label in labels:
    if label in subclass_to_class_int.keys():
        tensor_values.append(subclass_to_class_int[label])
    else:
        tensor_values.append(16)
        
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(tensor_keys, dtype=tf.int64),
        values=tf.constant(tensor_values,  dtype=tf.int64),
    ),
    default_value= tf.constant(0,  dtype=tf.int64)
)

def relabel_map(x, y):
    return x, table.lookup(y)

In [5]:
def resize(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    x = tf.image.resize(x, (224, 224))
    return x, y

def prepare_ds(ds, batch_size):
    dataset_mapped = ds.map(resize).map(relabel_map)
    dataset_size_after = 0
    for _, label in dataset_mapped:
        if label != 16:
            dataset_size_after += 1
    dataset_filtered = dataset_mapped.filter(lambda _, label : label != 16)
    dataset_filtered = dataset_filtered.apply(tf.data.experimental.assert_cardinality(dataset_size_after))
    dataset_prepared = dataset_filtered.shuffle(10000).batch(batch_size).prefetch(4)
    return dataset_prepared

def prepare_ds_without_batching(ds):
    dataset_mapped = ds.map(resize).map(relabel_map)
    dataset_size_after = 0
    for _, label in dataset_mapped:
        if label != 16:
            dataset_size_after += 1
    dataset_filtered = dataset_mapped.filter(lambda _, label : label != 16)
    dataset_filtered = dataset_filtered.apply(tf.data.experimental.assert_cardinality(dataset_size_after))
    dataset_prepared = dataset_filtered
    return dataset_prepared

# After the outsorting the other labels, the dataset became much smaller. To save the time during the dataset loading procedure, the created datasets are persisted locally

In [6]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    for ds, ds_name in zip([raw_valid_ds, raw_test_ds, raw_train_ds],['valid', 'test', 'train']):
        ds_without_batching = prepare_ds_without_batching(ds)
        tf.data.Dataset.save(ds_without_batching, "./saved_datasets/{0}_ds_without_batching".format(ds_name))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
